In [1]:
import PyPDF2
from nltk.corpus import stopwords

In [2]:
import nltk
import string
import re
from string import digits
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import CountVectorizer 
from stop_words import get_stop_words

from nltk.stem import PorterStemmer

In [3]:
def preprocessing(path):
    # open PDF
    pdf = PyPDF2.PdfFileReader(open(str(path),"rb"))
    
    # read PDF file in a list
    pdf_content = []
    for page in pdf.pages:
        pdf_content.append(page.extractText())
    pdf_content= str(pdf_content)
        
    #tokenisasi dan remove punctuation
    translator = str.maketrans('','',string.punctuation)
    new_pco = str()
    for i in range(len(pdf_content)):
        if pdf_content[i] == "n":
            if ord(pdf_content[i-1]) == 92:
                continue
            else:new_pco += pdf_content[i]
        elif ord(pdf_content[i]) == 92: continue
        else:new_pco += pdf_content[i]
    pdf_content = new_pco
    pdf_content = pdf_content.translate(translator)
    pdf_content = ''.join([i for i in pdf_content if not i.isdigit()])
    pdf_content = re.sub('r ^ https ?:\\/\\/.*[\\r\\n]*', '', pdf_content, flags=re.MULTILINE)
    pdf_content = word_tokenize(pdf_content)
    
    print ("Tokenisasi dan remove punctuation")
    print(pdf_content)
    print ("\n")
        
    #case folding
    caseFolding = [kata.lower() for kata in pdf_content]
    print ("Case folding")
    print(caseFolding)
    print ("\n")
    
    #remove stop word
    stopword_list = list(stopwords.words("indonesian"))
    without_stop_words = []
    for word in filter(None, caseFolding):
        if word not in stopword_list:
            without_stop_words.append(word)
    print ("Remove stop word")
    print(without_stop_words)
    print ("\n")
            
    #stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    kata_dasar=[]
    for kata in without_stop_words:
        kata_dasar.append(stemmer.stem(kata))
    print ("Stemming")
    print(kata_dasar)
    print ("\n")
    
    return kata_dasar

In [4]:
def preprocessingQuery(kata):  
    #case folding
    caseFolding = kata.lower()
    print ("Case folding")
    print(caseFolding)
    print ("\n")
    
    #tokenisasi dan remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    caseFolding = caseFolding.translate(translator)
    caseFolding = ''.join([i for i in caseFolding if not i.isdigit()])
    tokens = nltk.tokenize.word_tokenize(caseFolding)
    print ("Tokenisasi dan remove punctuation")
    print(tokens)
    print ("\n")
    
    #remove stop word
    stopword_list = list(stopwords.words("indonesian"))
    without_stop_words = []
    for word in filter(None, tokens):
        if word not in stopword_list:
            without_stop_words.append(word)
    print ("Remove stop word")
    print(without_stop_words)
    print ("\n")
    
    #stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    kata_dasar=[]
    for kata in without_stop_words:
        kata_dasar.append(stemmer.stem(kata))
    print ("Stemming")
    print(kata_dasar)
    print ("\n")
    
    return kata_dasar

In [5]:
from math import log

'''
Using the following formula to calculate BM25
((k3 + 1)q)/((k3 + q)) * ((k1 + 1)f)/((K + f)) * log((r + 0.5)(N − n − R + r + 0.5))/((n − r + 0.5)(R − r + 0.5))
REFERENCE: https://xapian.org/docs/bm25.html
'''

# DEFINING CONSTANTS

k1 = 1.2
b = 0.75
k2 = 100
R = 0 #Since no relevance info is available

# MAIN METHOD
def BM25(docLen, avDocLen, n, N, f, q, r):
    p1 = ((k2 + 1) * q) / (k2 + q)
    p2 = ((k1 + 1) * f) / (getK(docLen, avDocLen) + f)
    p3 = log((r + 0.5) * (N - n - R + r + 0.5)) / ((n - r + 0.5) * (R - r + 0.5))
    return p1 * p2 * p3

def getK(docLen, avDocLen):
    return k1 * ((1 - b) + b * (float(docLen) / float(avDocLen)))

In [6]:
import json
import operator
from collections import defaultdict
#from retrieval import BM25

# get average document length
def get_avdl(length_index):
    corpus_length = 0
    for document in length_index:
        corpus_length += length_index[document]
    return float(corpus_length) / float(len(length_index))

def search(query):
    inv_index_file = open("D:/KULIAH/Semester 8/Sistem Temu Balik Informasi/Proyek/Coba/inverted_index.json","r")
    inverted_index = json.load(inv_index_file)

    length_index_file = open("D:/KULIAH/Semester 8/Sistem Temu Balik Informasi/Proyek/Coba/length_index.json","r")
    length_index = json.load(length_index_file)

    scores = defaultdict(list)
    #query_tokens = query.split()
    #for token in query_tokens
    for token in query:
        for entry in inverted_index[token]:
            scores[entry[0]] = BM25(length_index[entry[0]],get_avdl(length_index),len(inverted_index[token]),len(length_index),entry[1],1,0)
    print (scores)
    print (scores[entry[0]])
    return sorted(scores.items(),key=operator.itemgetter(1))

In [7]:
from collections import defaultdict
import glob
import json


def get_file_names():
    files = []
    for file in glob.glob("D:/KULIAH/Semester 8/Sistem Temu Balik Informasi/Proyek/Coba/*.pdf"):
        files.append(file)
    return files


def make_index(tokens, document_name, index, length):
    for term in set(tokens):
        index[term].append([document_name,tokens.count(term)])
        length[document_name] = len(set(tokens))


def generator():
    files = get_file_names()
    inverted_index = defaultdict(list)
    length_index = defaultdict(list)
    for file in files:
        make_index(preprocessing(file), file, inverted_index, length_index)
    write(inverted_index,length_index)
    print ("Indexes generated")
    print (inverted_index)


def write(inverted_index,length_index):
    inv_index_file = open("D:/KULIAH/Semester 8/Sistem Temu Balik Informasi/Proyek/Coba/inverted_index.json","w")
    json.dump(inverted_index,inv_index_file)

    length_index_file = open("D:/KULIAH/Semester 8/Sistem Temu Balik Informasi/Proyek/Coba/length_index.json","w")
    json.dump(length_index,length_index_file)

In [ ]:
generator()
option = ""
while option != "q":
    print ("\n")
    print ("Enter search query")
    key = input(":: ")
    keywords = preprocessingQuery(key)
    results = search(keywords)
    print ("\nThe Matching Files Are:")
    for result in results:
        print (result[0])

Tokenisasi dan remove punctuation
['Potensi', 'Pengembangan', 'Industri', 'dan', 'Bioekonomi', 'Berbasis', 'Makanan', 'Fermentasi', 'Tradisional', 'SUYANTO', 'PAWIROHARSONOBadan', 'Pengkajian', 'dan', 'Penerapan', 'TeknologiJalan', 'MH', 'Thamrin', 'No', 'JakartaDiterima', 'Januari', 'Disetujui', 'Mei', 'Abstract', 'Consumption', 'of', 'traditionallyfermented', 'foods', 'is', 'around', 'of', 'the', 'total', 'human', 'food', 'consumption', 'Indonesia', 'has', 'several', 'important', 'traditionallyfermented', 'foods', 'such', 'as', 'tempe', 'fermented', 'soybean', 'tape', 'fermented', 'cassava', 'or', 'glutinous', 'rice', 'etc', 'Those', 'foods', 'are', 'in', 'general', 'produced', 'in', 'smallscale', 'industries', 'or', 'in', 'home', 'industries', 'which', 'appear', 'somewhat', 'dirty', 'unhygienic', 'and', 'not', 'standardized', 'Microorganisms', 'play', 'an', 'important', 'part', 'in', 'the', 'fermentation', 'process', 'The', 'microorganisms', 'involved', 'in', 'the', 'traditionallyfe

Stemming
['potensi', 'kembang', 'industri', 'bioekonomi', 'bas', 'makan', 'fermentasi', 'tradisional', 'suyanto', 'pawiroharsonobadan', 'kaji', 'terap', 'teknologijalan', 'mh', 'thamrin', 'no', 'jakartaditerima', 'januari', 'tuju', 'mei', 'abstract', 'consumption', 'of', 'traditionallyfermented', 'foods', 'is', 'around', 'of', 'the', 'total', 'human', 'food', 'consumption', 'indonesia', 'has', 'several', 'important', 'traditionallyfermented', 'foods', 'such', 'as', 'tempe', 'fermented', 'soybean', 'tape', 'fermented', 'cassava', 'or', 'glutinous', 'rice', 'etc', 'those', 'foods', 'are', 'in', 'general', 'produced', 'in', 'smallscale', 'industries', 'or', 'in', 'home', 'industries', 'which', 'appear', 'somewhat', 'dirty', 'unhygienic', 'and', 'not', 'standardized', 'microorganisms', 'play', 'an', 'important', 'part', 'in', 'the', 'fermentation', 'process', 'the', 'microorganisms', 'involved', 'in', 'the', 'traditionallyfermented', 'food', 'are', 'usually', 'a', 'multiculture', 'with', '

:: Kimia
Case folding
kimia


Tokenisasi dan remove punctuation
['kimia']


Remove stop word
['kimia']


Stemming
['kimia']


defaultdict(<class 'list'>, {'D:/KULIAH/Semester 8/Sistem Temu Balik Informasi/Proyek/Coba\\01. Potensi_Pengembangan_Industri_dan_Bioeko.pdf': -2.5415396620531325})
-2.5415396620531325

The Matching Files Are:
D:/KULIAH/Semester 8/Sistem Temu Balik Informasi/Proyek/Coba\01. Potensi_Pengembangan_Industri_dan_Bioeko.pdf


Enter search query
